# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")
items.rename(columns={'book_id': 'item_id'}, inplace=True)


In [4]:
als_recommendations = pd.read_parquet("als_recommendations.parquet")


# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [8]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx] # ваш код здесь #
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()# ваш код здесь #
# количество пользователей, которые есть и в train, и в test
common_users =  set(users_train) & set(users_test)# ваш код здесь #

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [7]:
# множества пользователей в train и test
set_train = set(users_train)
set_test = set(users_test)

# "холодные" пользователи — в test, но нет в train
cold_users_ids = set_test - set_train

# преобразуем в DataFrame, если нужно
cold_users = pd.DataFrame({'user_id': list(cold_users_ids)})

print(f'Число холодных пользователей: {len(cold_users)}')


Число холодных пользователей: 2365


In [8]:
from sklearn.preprocessing import MinMaxScaler

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

# нормализация пользователей и среднего рейтинга, требуется для их приведения к одному масштабу
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# вычисляем popularity_score, как скор популярности со штрафом за низкий рейтинг
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values(by='popularity_score', ascending=False)# ваш код здесь #

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"] > 4][:100]# ваш код здесь #
top_k_pop_items.head()

,item_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
30695,16096824,16770,4.301014,0.412126,0.825253,0.340108
1916,15881,13043,4.632447,0.320529,0.908112,0.291076


In [9]:
items.rename(columns={'book_id': 'item_id'}, inplace=True)

In [10]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


In [11]:
top_k_pop_items

,item_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score,author,title,genre_and_votes,publication_year
0,18007564,20207,4.321275,0.496596,0.830319,0.412333,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",2014
1,18143977,19462,4.290669,0.478287,0.822667,0.393471,Anthony Doerr,All the Light We Cannot See,"{'Historical-Historical Fiction': 13679, 'Fict...",2014
2,3,15139,4.706057,0.372042,0.926514,0.344702,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad...",1997
3,16096824,16770,4.301014,0.412126,0.825253,0.340108,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",2015
4,15881,13043,4.632447,0.320529,0.908112,0.291076,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict...",1999
...,...,...,...,...,...,...,...,...,...,...
95,8490112,4792,4.080968,0.117747,0.770242,0.090694,Laini Taylor,Daughter of Smoke & Bone (Daughter of Smoke & ...,"{'Fantasy': 11681, 'Young Adult': 7110, 'Roman...",2011
96,18966819,4361,4.374914,0.107154,0.843729,0.090409,Pierce Brown,"Golden Son (Red Rising, #2)","{'Science Fiction': 2613, 'Fantasy': 1372, 'Sc...",2015
97,3636,4667,4.098564,0.114675,0.774641,0.088832,Lois Lowry,"The Giver (The Giver, #1)","{'Young Adult': 10993, 'Fiction': 9045, 'Class...",2006
98,18293427,4674,4.092640,0.114847,0.773160,0.088795,Gabrielle Zevin,The Storied Life of A.J. Fikry,"{'Fiction': 3795, 'Contemporary': 1100, 'Writi...",2014


In [12]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users['user_id'])] \
    .merge(top_k_pop_items.set_index("item_id")[["avg_rating"]], on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]

In [13]:
cold_user_recs

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180
...,...,...,...,...
9637,1429316,9361589,2,4.085858
9638,1429316,18584855,4,4.071619
9641,1429720,9969571,4,4.290044
9644,1429798,11235712,5,4.179189


In [14]:

# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2))

0.78 0.62


In [15]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

In [16]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [17]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [18]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8263
MAE:  0.6460
0.826346375350908 0.6460143973270805


In [19]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [20]:
from surprise import accuracy

rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)
                     
print(rmse, mae)

RMSE: 1.2590
MAE:  0.9982
1.2590325375790072 0.9982060614713011


In [21]:
item_user_counts = events.groupby('item_id')['user_id'].nunique()
popular_items = item_user_counts[item_user_counts >= 3].index
events_filtered = events[events['item_id'].isin(popular_items)]
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events_filtered["started_at"] < train_test_global_time_split_date
events_ft_train = events_filtered[train_test_global_time_split_idx] # ваш код здесь #
events_ft_test = events_filtered[~train_test_global_time_split_idx]


In [22]:
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_ft_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)
surprise_test_set = list(events_ft_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [23]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8269
MAE:  0.6465
0.8268854479787304 0.6464733347343905


In [24]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])


result = get_recommendations_svd(1296647, items, events_test, svd_model)
result

,item_id,score
0,31422889,5.000000
1,31915456,4.950524
2,22037424,4.943957
3,481749,4.922973
4,18713259,4.916904


In [25]:
items.rename(columns={'book_id': 'item_id'}, inplace=True)

In [26]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

user_id: 1109853
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
0,Patrick Rothfuss,"The Wise Man's Fear (The Kingkiller Chronicle,...",2015-06-17,2015-08-31,5,"{'Fantasy': 16491, 'Fiction': 2222, 'Fantasy-E..."
1,Andy Weir,The Martian,2014-12-07,2014-12-11,4,"{'Science Fiction': 11966, 'Fiction': 8430}"
2,Brandon Sanderson,"The Way of Kings (The Stormlight Archive, #1)",2015-08-31,2015-10-30,5,"{'Fantasy': 14291, 'Fiction': 1623, 'Fantasy-E..."
3,Brandon Sanderson,"Words of Radiance (The Stormlight Archive, #2)",2015-10-30,2016-03-17,5,"{'Fantasy': 8542, 'Fiction': 872, 'Fantasy-Epi..."
4,Ken Follett,"Fall of Giants (The Century Trilogy, #1)",2016-05-12,2016-08-30,4,"{'Historical-Historical Fiction': 4665, 'Ficti..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,11221285,5.000000,Brandon Sanderson,"The Way of Kings, Part 2 (The Stormlight Archi...","{'Fantasy': 641, 'Fiction': 46, 'Fantasy-Epic ..."
1,24812,4.986302,Bill Watterson,The Complete Calvin and Hobbes,"{'Sequential Art-Comics': 867, 'Humor': 378, '..."
2,6219415,4.986246,"Ibrahim Nasrallah, إبراهيم نصر الله",زمن الخيول البيضاء,"{'Novels': 83, 'Fiction': 16, 'Historical-Hist..."
3,24814,4.980840,Bill Watterson,It's a Magical World: A Calvin and Hobbes Coll...,"{'Sequential Art-Comics': 680, 'Humor': 381, '..."
4,20451634,4.968815,Inio Asano,Oyasumi Punpun,"{'Sequential Art-Manga': 248, 'Sequential Art-..."


# === Базовые подходы: коллаборативная фильтрация

In [31]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])
srt = sorted(events_train['item_id_enc'], reverse=True)
srt[:5]

/tmp/ipykernel_4013/782784623.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_4013/782784623.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_4013/782784623.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

[43304, 43304, 43250, 43232, 43232]

In [28]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [29]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [30]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:17<00:00,  3.96s/it]


In [6]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

In [32]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [33]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [34]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")


In [9]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [36]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [37]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [38]:
print(ndcg_at_5_scores.mean())

0.975946709792109


# === Базовые подходы: контентные рекомендации

In [10]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [40]:
items

,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,text_reviews_count,publisher,publication_year,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc
3,6066819,Jennifer Weiner,Best Friends Forever,Addie Downs and Valerie Adler were eight when ...,"{'Womens Fiction-Chick Lit': 739, 'Fiction': 442}",368,3.49,51184,3282,Atria Books,2009,US,eng,Hardcover,False,0743294297,9780743294294,"{'Academic': None, 'Academic-Academia': None, ...","Womens Fiction-Chick Lit 739, Fiction 442",23133
6,378460,Michael Halberstam,The Wanting of Levine,,"{'Politics': 1, 'Humor': 1}",<NA>,4.38,12,4,Berkley Publishing Group,1979,US,,Paperback,False,0425040887,9780425040881,"{'Academic': None, 'Academic-Academia': None, ...","Politics 1user, Humor 1user",12687
15,89375,"Don Piper, Cecil Murphey",90 Minutes in Heaven: A True Story of Death an...,As he is driving home from a minister's confer...,"{'Christian': 395, 'Nonfiction': 392, 'Religio...",<NA>,3.91,68157,2885,,<NA>,US,,,False,0800759494,9780800759490,"{'Academic': None, 'Academic-Academia': None, ...","Christian 395, Nonfiction 392, Religion 142, S...",6460
16,89376,Randy Alcorn,Heaven,What is Heaven really going to be like? What w...,"{'Christian': 225, 'Religion-Theology': 154, '...",533,4.26,7345,566,,<NA>,US,eng,,False,0842379428,9780842379427,"{'Academic': None, 'Academic-Academia': None, ...","Christian 225, Religion-Theology 154, Nonficti...",6461
17,89377,Jennifer L. Holm,Penny from Heaven,It's 1953 and 11-year-old Penny dreams of a su...,"{'Historical-Historical Fiction': 284, 'Childr...",288,3.98,6949,615,Random House Books for Young Readers,2006,US,,Hardcover,False,037583687X,9780375836879,"{'Academic': None, 'Academic-Academia': None, ...","Historical-Historical Fiction 284, Childrens-M...",6462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2360257,279660,James Goldman,The Lion in Winter,Insecure siblings fighting for their parents' ...,"{'Plays': 294, 'Historical-Historical Fiction'...",103,4.22,8227,100,Random House,2004,US,eng,Paperback,False,0812973356,9780812973358,"{'Academic': None, 'Academic-Academia': None, ...","Plays 294, Historical-Historical Fiction 110, ...",11240
2360258,7657484,"J. Michael Straczynski, Shane Davis, Sandra Ho...","Superman: Earth One, Volume 1",Forget everything you know about The Man of St...,"{'Sequential Art-Comics': 683, 'Sequential Art...",134,3.90,13221,578,DC Comics,2010,US,eng,Hardcover,False,1401224687,9781401224684,"{'Academic': None, 'Academic-Academia': None, ...","Sequential Art-Comics 683, Sequential Art-Grap...",25871
2360322,7715664,Larissa Ione,Sin Undone (Demonica #5),HER TOUCH IS DEADLY\nAs the only female Seminu...,"{'Romance-Paranormal Romance': 703, 'Fantasy-P...",400,4.35,23091,819,Grand Central Publishing,2010,US,eng,Mass Market Paperback,False,0446556815,9780446556811,"{'Academic': None, 'Academic-Academia': None, ...","Romance-Paranormal Romance 703, Fantasy-Parano...",25962
2360395,30367813,J.A. Owenby,The Truth She Knew (The Truth Series #1),"""A bittersweet story of young love, independen...","{'Fiction': 4, 'Romance': 3, 'Business-Amazon'...",238,4.34,111,67,,2016,US,eng,Paperback,False,1533660549,9781533660541,"{'Academic': None, 'Academic-Academia': None, ...","Fiction 4, Romance 3, Business-Amazon 3",42220


In [11]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v.genre_and_votes
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes # ваш код здесь #
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [12]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres = genres.sort_values(by="score", ascending=False)
genres.head()

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926


In [33]:
import scipy
import sklearn.preprocessing
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr


In [34]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [45]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [46]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [47]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.195253
74,Fantasy-Magic,182982,0.096687
3,Humor,304302,0.074261
35,Romance-Paranormal Romance,311277,0.045902
5,Nonfiction,1737406,0.044359


In [48]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[-k:][::-1]
top_k_item_ids = item_encoder.inverse_transform(top_k_indices)
top_k_item_ids

array([ 50033,   9620, 516570,  49673,  16286])

In [49]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
564712,Ray Bradbury,"Farewell Summer (Green Town, #3)","{'Fiction': 170, 'Fantasy': 72, 'Science Fiction': 72, 'Classics': 52}"
1358935,John Fowles,The Magus,"{'Fiction': 1204, 'Classics': 421, 'Fantasy': 228, 'Mystery': 203, 'Literature': 167}"
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"


# === Базовые подходы: валидация

In [14]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [51]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=10)

/tmp/ipykernel_70359/3166051426.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [15]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [53]:
prec, rec = compute_cls_metrics(events_recs_for_binary_metrics)
print(prec, rec)

0.008732947582837622 0.03130238527136974


In [54]:
top_100 = als_recommendations[:100]
coverage_mask = items['item_id'].isin(als_recommendations['item_id'])
# расчёт покрытия по объектам
cov_items = coverage_mask.sum() / len(items)
print(f"{cov_items:.2f}") 

0.09


In [55]:
events_train

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,user_id_enc,item_id_enc
6679625,1000000,11012,2015-12-05,2015-12-11,True,4,False,0,1303
6679617,1000000,4671,2014-06-05,2014-06-30,True,5,False,0,493
6679618,1000000,5,2012-10-02,2012-10-24,True,5,False,0,3
6679620,1000000,2,2009-07-12,2009-07-29,True,5,False,0,1
6679621,1000000,14497,2016-05-09,2016-06-02,True,5,False,0,1808
...,...,...,...,...,...,...,...,...,...
5625381,1430584,29056083,2016-08-01,2016-08-03,True,3,True,430584,41809
5625379,1430584,25111004,2016-12-08,2016-12-08,True,5,False,430584,40432
5625378,1430584,6606855,2017-03-01,2017-03-01,True,3,False,430584,24391
5625377,1430584,18812405,2017-05-05,2017-05-31,True,3,True,430584,37138


In [16]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train[["user_id", "item_id","read"]], on=["user_id", "item_id"], how="left")
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values('score', ascending=False)
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
# ваш код здесь #

/tmp/ipykernel_4013/3046280952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["read"] = True


In [57]:
novelty_5.mean()

0.607333279143491

# === Двухстадийный подход: метрики

In [17]:
 # задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()

In [18]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=['user_id', "item_id"],
    how="outer")

In [21]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], how='left')
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])
candidates_for_train.head()

,user_id,item_id,als_score,cnt_score,target
615,1000006,29868610,0.286715,NaN,1
632,1000006,7445,0.230529,NaN,1
649,1000006,18812405,0.178382,NaN,1
1998,1000019,37415,0.043595,NaN,1
2302,1000023,7260188,0.598791,NaN,1


In [23]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 89.7ms	remaining: 1m 29s
100:	learn: 0.5118959	total: 2.23s	remaining: 19.9s
200:	learn: 0.5111710	total: 6.51s	remaining: 25.9s
300:	learn: 0.5105208	total: 9.24s	remaining: 21.5s
400:	learn: 0.5100174	total: 11.6s	remaining: 17.3s
500:	learn: 0.5095747	total: 13.5s	remaining: 13.5s
600:	learn: 0.5091600	total: 15.4s	remaining: 10.2s
700:	learn: 0.5087803	total: 17.3s	remaining: 7.38s
800:	learn: 0.5084220	total: 19.1s	remaining: 4.75s
900:	learn: 0.5080930	total: 21s	remaining: 2.3s
999:	learn: 0.5078081	total: 22.8s	remaining: 0us


# === Двухстадийный подход: модель

In [24]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=['user_id', "item_id"],
    how="outer")

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]
print(len(candidates_to_rank))

14517152


In [25]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]
final_recommendations.iloc[50]

user_id      1.000003e+06
item_id      4.214000e+03
als_score    3.764185e-01
cnt_score             NaN
cb_score     4.397070e-01
rank         5.100000e+01
Name: 368, dtype: float64

In [26]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.006, recall: 0.015


# === Двухстадийный подход: построение признаков

In [27]:
items["age"] = 2018 - items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

candidates_for_train2 = candidates_for_train.merge(items[['item_id', 'age', 'average_rating']], on='item_id', how='left')
candidates_to_rank2 = candidates_to_rank.merge(items[['item_id', 'age', 'average_rating' ]], on='item_id', how='left')

In [28]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max()-x.min()).days/365.25),
        books_read=('item_id', "count"),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std"))
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features
    
user_features_for_train = get_user_features(events_train)
candidates_for_train2 = candidates_for_train2.merge(user_features_for_train, on="user_id", how="left")
  
# оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank2 = candidates_to_rank2.merge(user_features_for_ranking, on="user_id", how="left")# ваш код здесь #

In [36]:
# определяем индексы топ-10 жанров и всех остальных
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]# ваш код здесь #

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres
    
user_genres_for_train = get_user_genres(events_train, items, genre_columns) # ваш код здесь #
candidates_for_train2 = candidates_for_train2.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank2 = candidates_to_rank2.merge(user_genres_for_ranking, on="user_id", how="left")

In [37]:
candidates_to_rank2

,user_id,item_id,als_score,cnt_score,cb_score,rank,age,average_rating,reading_years,books_read,...,genre_1,genre_38,genre_18,genre_34,genre_5,genre_16,genre_20,genre_24,genre_33,genre_others
0,1000003,49628,0.446143,0.906649,0.583617,1,14.0,4.01,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
1,1000003,7260188,1.129979,NaN,0.509005,2,8.0,4.03,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
2,1000003,6148028,1.123475,NaN,0.509005,3,9.0,4.30,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
3,1000003,2767052,1.112699,NaN,0.509005,4,10.0,4.34,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
4,1000003,43641,0.617602,NaN,0.477032,5,11.0,4.07,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14517147,1430580,183087,NaN,0.981247,0.066960,187,12.0,3.56,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901
14517148,1430580,33906,NaN,0.988908,0.064737,188,11.0,3.67,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901
14517149,1430580,82436,NaN,0.987934,0.064737,189,12.0,3.85,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901
14517150,1430580,82780,NaN,0.987217,0.061955,190,13.0,3.90,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901


In [ ]:
candidates_to_rank = candidates_to_rank[['user_id', 'item_id', 'als_score', 'cnt_score', 'cb_score', 'rank']].copy()

In [38]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train2[features], 
    label=candidates_for_train2[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6485043	total: 25.6ms	remaining: 25.6s
100:	learn: 0.4665268	total: 2.49s	remaining: 22.1s
200:	learn: 0.4578610	total: 4.93s	remaining: 19.6s
300:	learn: 0.4518428	total: 7.91s	remaining: 18.4s
400:	learn: 0.4471687	total: 10.8s	remaining: 16.2s
500:	learn: 0.4429055	total: 13.8s	remaining: 13.8s
600:	learn: 0.4390647	total: 16.3s	remaining: 10.8s
700:	learn: 0.4355651	total: 18.7s	remaining: 7.99s
800:	learn: 0.4321506	total: 21.2s	remaining: 5.26s
900:	learn: 0.4288350	total: 24.8s	remaining: 2.72s
999:	learn: 0.4257325	total: 27.8s	remaining: 0us


In [ ]:
inference_data = Pool(data=candidates_to_rank2[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank2["cb_score"] = predictions[:, 1]

# для каждого пользователя проставим rank, начиная с 1 — это максимальный cb_score
candidates_to_rank2 = candidates_to_rank2.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank2["rank"] = candidates_to_rank2.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank2.query("rank <= @max_recommendations_per_user")
final_recommendations.nunique()

,user_id,item_id,als_score,cnt_score,cb_score,rank,age,average_rating,reading_years,books_read,...,genre_1,genre_38,genre_18,genre_34,genre_5,genre_16,genre_20,genre_24,genre_33,genre_others
1,1000003,7260188,1.129979,NaN,0.969223,1,8.0,4.03,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
3,1000003,2767052,1.112699,NaN,0.962315,2,10.0,4.34,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
2,1000003,6148028,1.123475,NaN,0.951618,3,9.0,4.30,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
98,1000003,9361589,1.060634,NaN,0.574411,4,7.0,4.03,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
31,1000003,38447,0.537152,NaN,0.563894,5,20.0,4.07,7.4141,94.0,...,0.273534,0.04293,0.044853,0.013835,0.050522,0.09458,0.045009,0.04636,0.003504,0.316763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14517035,1430580,3798703,0.019319,NaN,0.184462,96,NaN,3.53,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901
14517054,1430580,34,0.016729,NaN,0.181927,97,15.0,4.34,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901
14517010,1430580,1618,0.023068,NaN,0.178909,98,NaN,3.85,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901
14517049,1430580,8852,0.017393,NaN,0.178872,99,NaN,3.88,0.0000,1.0,...,0.208679,0.00000,0.571420,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.219901


In [42]:
final_recommendations.to_parquet("final_recommendations_feat.parquet")


In [43]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=10
    )

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.011, recall: 0.054


In [46]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])
feature_importance = feature_importance.sort_values(by=['fi'], ascending=False)

print(feature_importance )

                       fi
als_score       30.043771
age             18.356837
average_rating  13.889881
books_read       7.140439
cnt_score        2.887655
reading_years    2.599439
genre_18         2.558679
genre_others     2.522204
genre_1          2.519999
genre_25         2.444803
genre_34         2.211371
books_per_year   1.740982
genre_38         1.588057
genre_20         1.582231
rating_avg       1.570182
genre_33         1.507410
genre_16         1.425483
genre_24         1.334677
genre_5          1.077762
rating_std       0.998138
